## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Constitucion,CL,-35.3333,-72.4167,60.96,64,0,9.86,clear sky
1,1,Baiyin,CN,36.5583,104.2081,42.80,93,75,8.95,light rain
2,2,Lavrentiya,RU,65.5833,-171.0000,32.47,74,29,0.96,scattered clouds
3,3,Yerbogachen,RU,61.2767,108.0108,27.82,87,95,6.49,overcast clouds
4,4,Guiyang,CN,26.5833,106.7167,71.60,78,75,4.47,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
4,4,Guiyang,CN,26.5833,106.7167,71.60,78,75,4.47,broken clouds
6,6,Andros Town,BS,24.7000,-77.7667,82.40,61,75,17.27,broken clouds
9,9,Manta,EC,-0.9500,-80.7333,80.60,78,75,6.91,broken clouds
10,10,Poum,NC,-20.2333,164.0167,78.55,79,92,7.99,overcast clouds
14,14,Rikitea,PF,-23.1203,-134.9692,79.09,67,100,17.78,overcast clouds
16,16,Lorengau,PG,-2.0226,147.2712,79.88,77,93,3.85,overcast clouds
20,20,Cockburn Town,TC,21.4612,-71.1419,82.40,65,40,9.22,scattered clouds
24,24,Arraial Do Cabo,BR,-22.9661,-42.0278,75.20,78,0,19.57,clear sky
26,26,Sao Joao Da Barra,BR,-21.6403,-41.0511,78.80,57,0,9.22,clear sky
28,28,Bubaque,GW,11.2833,-15.8333,78.33,71,2,17.36,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
# Check to see if new preferred_cities_df has any null values
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

# Check removal of null rows
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Guiyang,CN,71.60,broken clouds,26.5833,106.7167,
6,Andros Town,BS,82.40,broken clouds,24.7000,-77.7667,
9,Manta,EC,80.60,broken clouds,-0.9500,-80.7333,
10,Poum,NC,78.55,overcast clouds,-20.2333,164.0167,
14,Rikitea,PF,79.09,overcast clouds,-23.1203,-134.9692,
16,Lorengau,PG,79.88,overcast clouds,-2.0226,147.2712,
20,Cockburn Town,TC,82.40,scattered clouds,21.4612,-71.1419,
24,Arraial Do Cabo,BR,75.20,clear sky,-22.9661,-42.0278,
26,Sao Joao Da Barra,BR,78.80,clear sky,-21.6403,-41.0511,
28,Bubaque,GW,78.33,clear sky,11.2833,-15.8333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")


# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))